In [1]:
import SimpleITK as sitk
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import os
import numpy as np


refMask_path = './Reference/Reference_Mask.png'
ref_path = './Reference/Reference.png'
input_path = './Masks'
masks_dest = 'Masks_Registered_with_SimpleElastix'
pngs = './Pngs'
out = './Registered_with_Transformix'


In [2]:
# creates a new folder if it does not already exist
def new_dir(dirName):
    try:
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")

# target and reference are assumed as sitk.Image arrays
def affine_registration(fixed, moving):
    
    parameterMap = sitk.GetDefaultParameterMap('affine')
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixedImage)
    elastixImageFilter.SetMovingImage(movingImage)
    elastixImageFilter.SetParameterMap(parameterMap)
    elastixImageFilter.Execute()
    resultImage = elastixImageFilter.GetResultImage()
    transformParameterMap = elastixImageFilter.GetTransformParameterMap()
    return resultImage, transformParameterMap

def apply_transform(parameterMap, img):
    transformixImageFilter = sitk.TransformixImageFilter()
    transformixImageFilter.SetTransformParameterMap(parameterMap)
    transformixImageFilter.SetMovingImage(img)
    transformixImageFilter.Execute()
    resultImage = transformixImageFilter.GetResultImage()
    return resultImage
        

def IoU(img1, img2):
    intersection = np.logical_and(img1, img2)
    union = np.logical_or(img1, img2)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score
    
  
  

In [3]:
%%time
fixedImage = sitk.ReadImage(refMask_path, sitk.sitkFloat32)
for filename in os.listdir(input_path):
    if filename.endswith(".png"):
        print(filename)
        path = os.path.join(input_path, filename)
        movingImage = sitk.ReadImage(path, sitk.sitkFloat32)
        path2 = os.path.join(masks_dest, filename) 
        ri, tpm = affine_registration(fixedImage, movingImage)
        path3 = os.path.join(pngs, filename)
        img = sitk.ReadImage(path3.replace('_Mask.png', '.png'), sitk.sitkFloat32)
        k = apply_transform(tpm, img)
        org = sitk.GetArrayFromImage(img)
        tr = sitk.GetArrayFromImage(k)
        res = sitk.GetArrayFromImage(ri)
        ret, res = cv2.threshold(res, 120, 255, cv2.THRESH_BINARY)
        path4 = os.path.join(out, filename)
        
        result1 = Image.fromarray(res)
        result1.convert('P').save(path2.replace('.png','_SimpleElastix.png'), 'PNG')
        result2 = Image.fromarray(tr)
        result2.convert('P').save(path4.replace('_Mask.png','_Transformix.png'), 'PNG')
    else:
        continue


000434271f63a053c4128a0ba6352c7f_Mask.png
00053190460d56c53cc3e57321387478_Mask.png
0005e8e3701dfb1dd93d53e2ff537b6e_Mask.png
0006e0a85696f6bb578e84fafa9a5607_Mask.png
0007d316f756b3fa0baea2ff514ce945_Mask.png
000ae00eb3942d27e0b97903dd563a6e_Mask.png
000d68e42b71d3eac10ccc077aba07c1_Mask.png
00150343289f317a0ad5629d5b7d9ef9_Mask.png
00176f7e1b1cb835123f95960b9a9efd_Mask.png
001d127bad87592efe45a5c7678f8b8d_Mask.png
0021df30f3fddef551eb3df4354b1d06_Mask.png
00291f7aff0123ea76a59998effef229_Mask.png
002a34c58c5b758217ed1f584ccbcfe9_Mask.png
0032c6091dc8f1b1245fc2f5f45458fa_Mask.png
003cfe5ce5c0ec5163138eb3b740e328_Mask.png
0046f681f078851293c4e710c4466058_Mask.png
004d2bc2111d639f5e8441ced52d55cb_Mask.png
004dc2a50591fb5f1aaf012bffa95fd9_Mask.png
004f33259ee4aef671c2b95d54e4be68_Mask.png
00575e3846ebd05a909d97ba59c53d30_Mask.png
0059d21bef1793fa9522e4ec8cae1a1a_Mask.png
005be26a68485912e007a3703f43d60a_Mask.png
005d70155f949c7785671800f2c8e1ca_Mask.png
0061cf6d35e253b6e7f03940592cc35e_M

0304212405f7474d54f89d76f3d8912b_Mask.png
031625d57b5e9a5c8bcd7b9a82246fcb_Mask.png
031f23b4a8658fe9b4fc18007cc95851_Mask.png
032111ac5bb2e0cbb9a22745a77eef8f_Mask.png
0325dcb3ad5f33f107e850fcdd6e5b4f_Mask.png
032993869b3b33cdc46ec374855c19a0_Mask.png
032c81fae916e932002387494a7d7c7b_Mask.png
0335eb5b09f03f1a943d5335d7fa129d_Mask.png
0339aac16e3dd93ee397315a224147a6_Mask.png
033bc3846ebbf668b702a5719f966fb5_Mask.png
033d34dc31e0ee291d11af204c9376d5_Mask.png
033dae57cec0aca171d47090f299bed2_Mask.png
033e1637bed9b9f3dccac9c6c419adc2_Mask.png
033ff76227ffb160868813782a3c3a1a_Mask.png
03431b577d1ccf075e930c4c4913c079_Mask.png
0343e7f7e9e4cc1ac974ab130d46eafa_Mask.png
0344c63913224cf62e3d8bcd5dd93a9b_Mask.png
034814210bf2105ecb3e7ed6876f43e9_Mask.png
034b98d64dc012298afb3d33fe880193_Mask.png
034cf2b503a9a7efe404f000fb988534_Mask.png
035480fbf46e946e21e7dce78637c329_Mask.png
035b53e8aabf8763537a296c291135d1_Mask.png
035dcbac2fa931002bde1f9853c4a30a_Mask.png
036161ca6aa4daa43cc3bc55602e8960_M

05b42653897fa9739366a25612f1eb08_Mask.png
05ba77b1cf87cd4597d988581f269b44_Mask.png
05c0c802db4f100de8e2a9fa4aa62d14_Mask.png
05c9d4438807f4ecbbea5be0c60e6ec6_Mask.png
05ca9b3be8bc8240cef9a387a5a06785_Mask.png
05cf1a6a9865043c51e6155474f11f81_Mask.png
05d1b84871feea216485eb3ad14633d8_Mask.png
05d3315aef94ddf37fe9c4886816a702_Mask.png
05d45c0a96b2c363f8b9a34a1f181c58_Mask.png
05d676834dbed1639cb5eea70c1e307b_Mask.png
05d92b99575f75e64e0c0e7293c13803_Mask.png
05dbd765ff84cbb24759b0c135ba4447_Mask.png
05dc427060325263843c57a04aeee7ed_Mask.png
05dfd35f424da60897dfafc2340e0891_Mask.png
05e0b2465053b685ca37849eac3dd62f_Mask.png
05e1efa511adb8c1e2631b13734031d1_Mask.png
05e560d65da399f4310a51c5bc7f44d8_Mask.png
05e64c5b1e5f246ae6e8bc109e557bf4_Mask.png
05e951c63e80999f13e6e09e7ec8439a_Mask.png
05ea0fb3ee5e5ca0b523e1fdf1a883d6_Mask.png
05ebb3c5e091a8322a50672d478d117e_Mask.png
05ee3741c4523f2265a0dc3c1b69d529_Mask.png
05f202ee95a17a4b1e6dbe1916ffe8bf_Mask.png
05f3c5986c9c49c6f04d4e15fa01a785_M

08c182c884fec1fad1b1e4a19d482acf_Mask.png
08ca5715d79a32bb574d28b34c0ed052_Mask.png
08cad5510452011e9c519bbbc9741fda_Mask.png
08ce56202e44175674fc5f5517e74db4_Mask.png
08d0eab34ea5bab14e7e56dabafcdb7f_Mask.png
08d348d8be4861efbf70a92ca86b8465_Mask.png
08d6125cda5ffceb8f235c96179a7061_Mask.png
08daf3cf9bcf3d306ad08dbb9a53f4f2_Mask.png
08e3d3783927a15d35137c756c1a7e0b_Mask.png
08e4edc0d35455c8e25a74b9ed3137b0_Mask.png
08eae737d6de0db70d4bcb02b799cd02_Mask.png
08ee53ea1825d6e2feffc2a6488f2d7d_Mask.png
08f362b8c0da181ca414c58e52555c64_Mask.png
08f4d0d827ef5216c35362812eeae644_Mask.png
08f84015dd3811451c7a99f3357480be_Mask.png
0900f93acb9f42caa354036b6453816d_Mask.png
0902a07255f28d5d15a24214761c743c_Mask.png
0905c0dc82fb8fe621c43f53719a55ca_Mask.png
0906a09b04f8d82c2cad5b820602a403_Mask.png
090a90ca6e01ed9f613578c95c151a01_Mask.png
090b30ca55ab2a592cd3b24c9407a2bf_Mask.png
090c217ff0664a8cfc1a382cacad32bb_Mask.png
09125244cb0f0db65b66ce7f33fa02c8_Mask.png
09175449df9374e0f6afe517f7b6249f_M

0b69b0af56333c2aa5afdce741f50ad9_Mask.png
0b6c10e0cd5868289d11c20da87410b8_Mask.png
0b78363604f4d81c3e27f90bd3066df6_Mask.png
0b7995c5e532803dc1bd077eaa4d2a01_Mask.png
0b7b8344c7c39373cfddf21a7339f206_Mask.png
0b80286547650e8b2686d9a8b8fcc5b1_Mask.png
0b805259394307f01adc87b57b704602_Mask.png
0b80aaae33916b6c56b4308584cd67c0_Mask.png
0b817af78ed58aad2612c0211b5878ec_Mask.png
0b877b934aada3284ac13cfa74b5419c_Mask.png
0b8c73a84bc8b5f1a11b27619c1bd8a3_Mask.png
0b8ec8f17db23936e86de9ca3f36d206_Mask.png
0b98b21145a9425bf3eeea4b0de425e7_Mask.png
0b9a96125c03add5610316081f13aa2e_Mask.png
0b9dd99214ab1ef052cdfd62fdfe82c4_Mask.png
0b9f58b6f12a231738723f4d2dda0312_Mask.png
0ba74ac32c808042eb0044c4871c8562_Mask.png
0bb76fb5f7cfb1dd4c3760f7481a1d05_Mask.png
0bd02888a92ad8428c8ec8ed971eab12_Mask.png
0bd8e34912a6344fc3962b39d6e46a7e_Mask.png
0bdadb23e846f9c247d3ea571f63913d_Mask.png
0bdb1109dfa8f6f0f3a8c4fad249506c_Mask.png
0bdf902cc4ac4ad2c49059960f572ab2_Mask.png
0be2bdac4e84982ad1b24830d2cce470_M

...

f1c8c1427d797062d35457c3d54bce52_Mask.png
f1c94b69b9f1468a758cf2a8b5825716_Mask.png
f1cbcf9205d98d491f92f5aec2cd08a5_Mask.png
f1cc23ec4bcfdb2cea60a249d3ecae6d_Mask.png
f1cc6ecd5562c49f2c25d539722fa074_Mask.png
f1d1d044621a14cf86416b00c84139dc_Mask.png
f1d1e5089e66fc256f08e621b5dcc9bf_Mask.png
f1d5d21120f5370c55efc2ab3d2db8bc_Mask.png
f1da5e509f8a55be3ac43a1b0034ca77_Mask.png
f1dd9b481aae4ba412e5fbea06215f53_Mask.png
f1dfc3bfb919da31d848eda5a20a8258_Mask.png
f1e06b99689d0e62ee0d9967a2c9d711_Mask.png
f1e4056384e1926c3ae33882e0dc00cb_Mask.png
f1e4ac8d6db9d986fbff411c6a314857_Mask.png
f1e7feabd0d43dc3d823995002eb6cd7_Mask.png
f1eb2216d773ced6330b1f31e18f04f8_Mask.png
f1ece4212015e746f3d90a95feb17546_Mask.png
f1ed2d380bf23214b0522e753035545c_Mask.png
f1f2e5d1b29b69b45979a8c295c20387_Mask.png
f1f4482fe3a8ede769d6c6c4bca3d0df_Mask.png
f1f76abbb54845f5be539cb4f8298a51_Mask.png
f1fd27cbbce1593e6b8c9c2c1e168e26_Mask.png
f20a7f9e51501b58856fdca5421f360a_Mask.png
f21b875faf5d86d8433c8590348e92e3_M

f4c12a07224893104dec25b926db33a1_Mask.png
f4c3a7a4453fe68b3868bd47b08f5050_Mask.png
f4c75572b1b62a80c3b1d29bd01f3ce6_Mask.png
f4c7a984e4c5c33174f6132f47a3a6f7_Mask.png
f4c97bfedbc71ab64cf312ea2962971e_Mask.png
f4ca8381763f4039d005b06fd8d662ca_Mask.png
f4cd256e9ac554106d34aed35ce8943b_Mask.png
f4d47aa68af0a6a3b5ca91e721707421_Mask.png
f4d61d727424622b8c54212967467c32_Mask.png
f4d7a04f379295bb190b0c95567f7e0a_Mask.png
f4d7d5de6fbfafb222c389fb23251e80_Mask.png
f4da0959856bcdd4b91e0c6949b109e3_Mask.png
f4dbbcc82af4455c1cd4411d37ef28c6_Mask.png
f4de2c4362a04f072fa06a8440bc7473_Mask.png
f4e16cb73fddfba9a3f054fc3d3fdcef_Mask.png
f4e19d9d1a1dba4f7491ac88bf894523_Mask.png
f4e77a5ddc822faaf55570590689466b_Mask.png
f4e8dbd6273e7bfea54ca8fcabc17ead_Mask.png
f4e913c031f50fa859d8c775cf716974_Mask.png
f4ef27a5e6ff0608dd0732509f077d98_Mask.png
f4f673fae6b00cc4c2210882b73555b3_Mask.png
f4f6c2e1ce48c49b19d62c3a12d9d6c0_Mask.png
f4fb761df33e2babd7d7fdab4dcb78fc_Mask.png
f4fcacd5c7e9d885ca45382b35797561_M

f774dada729622b4f5e2e1c352169d77_Mask.png
f77548b76082f9b45c05764493005039_Mask.png
f777a3ef5831c6a7a21d5a6be937beb7_Mask.png
f7793fee61ea241a3fc07b927e4914f6_Mask.png
f77a11e4afd7871bd5dbec2d11451863_Mask.png
f77c69a7b261fe9a5dfb469271f7d5cb_Mask.png
f78a273e30b5cd8a2d580fa4959c8433_Mask.png
f78a44efd6e19fe5e6f71247a4d97126_Mask.png
f78abb1dbaf45a2dce966e9d3877b78f_Mask.png
f78c70190e569c970a721b97152eed1a_Mask.png
f78dd290c9f2df8a451145858b4966c5_Mask.png
f78e1c855e003640a3d79723f16f5f1e_Mask.png
f791a58edd0f0080354090193069373d_Mask.png
f79600688fab0a766bc086be549bf78b_Mask.png
f7975e0d3f833ad7f395b1c895db6189_Mask.png
f7987e0db0dcf873c778b099ebd9bf1a_Mask.png
f799e015965be51263840fee9a2fd37d_Mask.png
f79c3fc269e1b961c4930926e32053c0_Mask.png
f7a3e5810397de8050fe067a5534bf2c_Mask.png
f7a56f47ad3ca3e1a06fd41bba7a3b03_Mask.png
f7b1e8842f18b17265754367a8d92ee6_Mask.png
f7b3414f5897f955261269f1f239c2f0_Mask.png
f7b383025dbf8471573457bcdb714d84_Mask.png
f7b3fa818839a04913e45fa796203462_M

fa7b469e21cf7a4e7afe9f17cf721259_Mask.png
fa7d454b6cb43448ed5a8da49df6ce05_Mask.png
fa8115db4830d2b29eccf4f133341a7c_Mask.png
fa86f62070ecf5d19f8d8662215f966f_Mask.png
fa8a4206a96b99a55f7e8c84a1ec5892_Mask.png
fa8a619dcb85f74699af1582fdb5d80a_Mask.png
fa8fe6da7a88296fdd66fcfd8b2729d1_Mask.png
fa93ce20869f64a1d44d6b8b3d962061_Mask.png
fa9b00efe5d0ecb21d10219151b95c56_Mask.png
fa9cd1a78a6d2d8d6e918a8c4125faf4_Mask.png
fab12e673d13db9d1daeb4ae01255cd1_Mask.png
fab2ad94eb5bac3d71e0046a9787579a_Mask.png
fab76d60c7fad833d0dccaccda261c3a_Mask.png
fabac1af1db73f2ee503f521e9659b72_Mask.png
fabc40c23f1169f51c9fe6cec6dfd08b_Mask.png
fac75d8df144493c44c741ccb0efba3d_Mask.png
face66ab3d09010b917e60958a63efbb_Mask.png
face7d0feea230cb057697e34fcb3ad3_Mask.png
fad39dc356aaa2da58470c6daaba8112_Mask.png
fad4bd66087b5d10e190636698e66c2a_Mask.png
fad7d9b912b9cd36ea45ba0c4ac51060_Mask.png
fada1146da6a0a81a9836fed22c9ae67_Mask.png
fae0a817c65ac6e549e94ac021544c22_Mask.png
fae3d9c7c5d474bb9cac8ac1c8912688_M

fd4b7dcfd91bdd1a5b51e96e6e0fe931_Mask.png
fd536e3e8f6f2336f27c76360c183b72_Mask.png
fd5c6d70e4e8fdf6a27d45d42605be45_Mask.png
fd6032971f08b0246235cd6c3f09fbae_Mask.png
fd6258ff19301497a66a19dfbe788b3c_Mask.png
fd63afac528fb70393d1ca8f442427c4_Mask.png
fd66c489f1de56737aabeb8838227f15_Mask.png
fd6b274f9e2ef0f660d52ed947e58ac3_Mask.png
fd6d801a9b661fd0abe97e5c37567c2b_Mask.png
fd709b976ae14089b17bea48208573d8_Mask.png
fd78c7d8b42f7d4c6c12e1ab79efbc65_Mask.png
fd80612e19d0c248b8477125cba4a350_Mask.png
fd810298e165ef0b9a88bb25fda7a34b_Mask.png
fd82f322cd3d32ba2f34959530b8a2df_Mask.png
fd8af9c6f7a66a823f719ec20f2412b0_Mask.png
fd8b4cd2cef008b9fff719a170e9b32d_Mask.png
fd8bd7efba38113c2848079bab15f193_Mask.png
fd8dac9e53275d18e40ce4aa1666bc7e_Mask.png
fd963d879cc51028141fe77ee665856a_Mask.png
fd984c0e930169a20e6a2d02e1dab204_Mask.png
fd992e43827acccf60702fe006c48399_Mask.png
fd9e829db017bf7d3750a815b2f34e89_Mask.png
fd9ea90f51379923e450f7578cb032ee_Mask.png
fda0ab589326a627136a6b9f0b4cc21c_M